# CPAS PEP Query

In [1]:
# Connecting to CPAS with cx_Oracle
# cx_Oracle is a Python extension module that enables access to Oracle Database
import cx_Oracle
import sqlalchemy
import os
import urllib
import pandas as pd
USERNAME = os.environ.get('CPAS_USERNAME')
PASSWORD = os.environ.get('CPAS_PASSWORD')
HOSTNAME = '10.32.196.224'
PORT = '1332'
SERVICE_NAME = 'visispd1.lacity.org'

In [2]:
dsn = cx_Oracle.makedsn(HOSTNAME, PORT, service_name=SERVICE_NAME)
connection = cx_Oracle.connect(user=USERNAME, password=PASSWORD, dsn=dsn)
import sqlalchemy

In [3]:
engine = sqlalchemy.create_engine(
f"oracle+cx_oracle://{USERNAME}:{PASSWORD}@{HOSTNAME}:{PORT}/"
f"?service_name={SERVICE_NAME}"
)

In [4]:
USER = "GRNT44"
engine.execute(f"SELECT table_name FROM all_tables WHERE owner='{USER}'").fetchall()

df = pd.read_sql(
    """SELECT
    '44' AS pgm_year,
    grnt_pep_id,
    pep_agcy_nm,
    pep_proj_nm,
    citywide_cncl_dist_yn,
    grnt_l_pep_stts,
    to_char(pep_vrftn_dt, 'mm/dd/yyyy') AS pep_vrftn_dt,
    to_char(pep_to_environ_dt, 'mm/dd/yyyy') AS pep_to_environ_dt,
    to_char(pep_from_environ_dt, 'mm/dd/yyyy') AS pep_from_environ_dt,
    to_char(pep_to_mgmt_dt, 'mm/dd/yyyy') AS pep_to_mgmt_dt,
    to_char(pep_send_out_dt, 'mm/dd/yyyy') AS pep_send_out_dt,
    grnt_l_send_out_mthd,
    to_char(cdbo_send_vrftn_dt, 'mm/dd/yyyy') AS cdbo_send_vrftn_dt,
    to_char(pep_rcv_dt, 'mm/dd/yyyy') AS pep_rcv_dt,
    pep_rcv_dt AS pep_rcv_dt_2,
    (
        SELECT
            field_value
        FROM
            grnt44.grnt_l_pep_typ
        WHERE
            field_id = gp.grnt_l_pep_typ
    ) AS grnt_l_pep_typ_value,
    (
        SELECT
            prjct_ttl
        FROM
            grnt44.grnt_aplctn
        WHERE
            grnt_aplctn_id = gp.grnt_aplctn_id
    ) AS prjct_ttl,
    cdbg_fnd_amt,
    (
        SELECT
            proj_id
        FROM
            grnt44.grnt_prpsd_proj
        WHERE
            grnt_aplctn_id = gp.grnt_aplctn_id
    ) AS proj_id,
    (
        SELECT
            pep_note
        FROM
            grnt44.grnt_pep_note
        WHERE
            grnt_l_pep_note_typ = 470
            AND grnt_pep_id = gp.grnt_pep_id
    ) AS trackingcomment,
    (
        SELECT
            pep_note
        FROM
            grnt44.grnt_pep_note
        WHERE
            grnt_l_pep_note_typ = 485
            AND grnt_pep_id = gp.grnt_pep_id
    ) AS pep_vrftn_comment,
    (
        SELECT
            pep_note
        FROM
            grnt44.grnt_pep_note
        WHERE
            grnt_l_pep_note_typ = 486
            AND grnt_pep_id = gp.grnt_pep_id
    ) AS pep_to_environ_comment,
    (
        SELECT
            pep_note
        FROM
            grnt44.grnt_pep_note
        WHERE
            grnt_l_pep_note_typ = 422169
            AND grnt_pep_id = gp.grnt_pep_id
    ) AS pep_from_environ_comment,
    (
        SELECT
            pep_note
        FROM
            grnt44.grnt_pep_note
        WHERE
            grnt_l_pep_note_typ = 487
            AND grnt_pep_id = gp.grnt_pep_id
    ) AS pep_to_mgmt_comment,
    (
        SELECT
            pep_note
        FROM
            grnt44.grnt_pep_note
        WHERE
            grnt_l_pep_note_typ = 488
            AND grnt_pep_id = gp.grnt_pep_id
    ) AS cdbo_send_vrftn_comment,
    (
        SELECT
            field_value
        FROM
            grnt44.grnt_l_pep_stts
        WHERE
            field_id = gp.grnt_l_pep_stts
    ) AS grnt_l_pep_stts_value,
    (
        SELECT
            field_value
        FROM
            grnt44.grnt_l_send_out_mthd
        WHERE
            field_id = gp.grnt_l_send_out_mthd
    ) AS grnt_l_send_out_mthd_value,
    (
        SELECT
            ldp.field_value
        FROM
            grnt44.grnt_aplctn   apl,
            grnt44.grnt_cntct    cicon,
            grnt.grnt_l_dept     ldp
        WHERE
            apl.grnt_aplctn_id = gp.grnt_aplctn_id
            AND cicon.cty_dept_yn = 1
            AND apl.grnt_city_dept_cntct_id = cicon.grnt_cntct_id
            AND cicon.grnt_l_dept = ldp.field_id
    ) AS department,
    grnt_l_rec_color,
    (
        SELECT
            field_value
        FROM
            grnt44.grnt_l_rec_color
        WHERE
            field_id = gp.grnt_l_rec_color
    ) AS grnt_l_rec_color_value,
    (
        SELECT
            grnt_user.grnt_utility.value_list('
                        select 
                            field_value 
                        from 
                            GRNT44.GRNT_PEP_CNCL_DIST A,
                            GRNT44.GRNT_L_CNCL_DIST B

                        where B.FIELD_ID=A.GRNT_L_CNCL_DIST
                        and
                        grnt_pep_id = '
            || gp.grnt_pep_id)
        FROM
            dual
    ) AS council_district,
    temp.status_date
FROM
    grnt44.grnt_pep gp,
    (
        SELECT
            MAX(mod_dt) AS status_date,
            pk_id
        FROM
            grnt38.grnt_hist
        WHERE
            table_nm = 'GRNT_PEP'
            AND pk_nm = 'GRNT_PEP_ID'
            AND field_nm = 'GRNT_L_PEP_STTS'
        GROUP BY
            pk_id
    ) temp
WHERE
    gp.grnt_pep_id = temp.pk_id (+)
    AND grnt_pep_id IN (
        SELECT
            grnt_pep_id
        FROM
            grnt44.grnt_pep
    )
ORDER BY
    proj_id,
    pep_agcy_nm""", engine)

df.head(5)

,pgm_year,grnt_pep_id,pep_agcy_nm,pep_proj_nm,citywide_cncl_dist_yn,grnt_l_pep_stts,pep_vrftn_dt,pep_to_environ_dt,pep_from_environ_dt,pep_to_mgmt_dt,...,pep_from_environ_comment,pep_to_mgmt_comment,cdbo_send_vrftn_comment,grnt_l_pep_stts_value,grnt_l_send_out_mthd_value,department,grnt_l_rec_color,grnt_l_rec_color_value,council_district,status_date
0,44,488681,1736 FAMILY CRISIS CENTER,DOMESTIC VIOLENCE SHELTER OPERATIONS,1.0,530,07/05/2018,07/05/2018,07/10/2018,07/12/2018,...,None,None,None,Approved,None,HOUSING AND COMMUNITY INVESTMENT DEPARTMENT,4.0,Green,None,None
1,44,488507,1736 FAMILY CRISIS CENTERS,FAMILYSOURCE CENTERS (NON PROFIT MANAGED),0.0,530,06/15/2018,06/15/2018,06/18/2018,06/20/2018,...,None,None,None,Approved,None,HOUSING AND COMMUNITY INVESTMENT DEPARTMENT,4.0,Green,10,None
2,44,487751,AG PACIFIC CONSTRUCTION & DEVELOPMENT CORP.,URGENT REPAIR PROGRAM,1.0,530,05/08/2018,05/08/2018,05/16/2018,05/17/2018,...,None,None,None,Approved,None,HOUSING AND COMMUNITY INVESTMENT DEPARTMENT,4.0,Green,None,None
3,44,490739,AGING DEPARTMENT,AGING DEPARTMENT,0.0,530,10/24/2018,10/24/2018,11/26/2018,10/31/2018,...,None,None,None,Approved,None,"AGING, DEPARTMENT OF",4.0,Green,14,None
4,44,488477,ALL PEOPLE'S COMMUNITY CENTER,FAMILYSOURCE CENTERS (NON-PROFIT MANAGED),0.0,530,06/15/2018,06/15/2018,06/18/2018,06/21/2018,...,None,None,None,Approved,None,HOUSING AND COMMUNITY INVESTMENT DEPARTMENT,4.0,Green,9,None


In [5]:
USER = "GRNT41"
engine.execute(f"SELECT table_name FROM all_tables WHERE owner='{USER}'").fetchall()

df = pd.read_sql(
    """SELECT
    '41' AS pgm_year,
    grnt_pep_id,
    pep_agcy_nm,
    pep_proj_nm,
    citywide_cncl_dist_yn,
    grnt_l_pep_stts,
    to_char(pep_vrftn_dt, 'mm/dd/yyyy') AS pep_vrftn_dt,
    to_char(pep_to_environ_dt, 'mm/dd/yyyy') AS pep_to_environ_dt,
    to_char(pep_to_mgmt_dt, 'mm/dd/yyyy') AS pep_to_mgmt_dt,
    to_char(pep_send_out_dt, 'mm/dd/yyyy') AS pep_send_out_dt,
    grnt_l_send_out_mthd,
    to_char(cdbo_send_vrftn_dt, 'mm/dd/yyyy') AS cdbo_send_vrftn_dt,
    to_char(pep_rcv_dt, 'mm/dd/yyyy') AS pep_rcv_dt,
    (
        SELECT
            field_value
        FROM
            grnt41.grnt_l_pep_typ
        WHERE
            field_id = gp.grnt_l_pep_typ
    ) AS grnt_l_pep_typ_value,
    (
        SELECT
            prjct_ttl
        FROM
            grnt41.grnt_aplctn
        WHERE
            grnt_aplctn_id = gp.grnt_aplctn_id
    ) AS prjct_ttl,
    cdbg_fnd_amt,
    (
        SELECT
            proj_id
        FROM
            grnt41.grnt_prpsd_proj
        WHERE
            grnt_aplctn_id = gp.grnt_aplctn_id
    ) AS proj_id,
    (
        SELECT
            pep_note
        FROM
            grnt41.grnt_pep_note
        WHERE
            grnt_l_pep_note_typ = 470
            AND grnt_pep_id = gp.grnt_pep_id
    ) AS trackingcomment,
    (
        SELECT
            pep_note
        FROM
            grnt41.grnt_pep_note
        WHERE
            grnt_l_pep_note_typ = 485
            AND grnt_pep_id = gp.grnt_pep_id
    ) AS pep_vrftn_comment,
    (
        SELECT
            pep_note
        FROM
            grnt41.grnt_pep_note
        WHERE
            grnt_l_pep_note_typ = 486
            AND grnt_pep_id = gp.grnt_pep_id
    ) AS pep_to_environ_comment,
    (
        SELECT
            pep_note
        FROM
            grnt41.grnt_pep_note
        WHERE
            grnt_l_pep_note_typ = 422169
            AND grnt_pep_id = gp.grnt_pep_id
    ) AS pep_from_environ_comment,
    (
        SELECT
            pep_note
        FROM
            grnt41.grnt_pep_note
        WHERE
            grnt_l_pep_note_typ = 487
            AND grnt_pep_id = gp.grnt_pep_id
    ) AS pep_to_mgmt_comment,
    (
        SELECT
            pep_note
        FROM
            grnt41.grnt_pep_note
        WHERE
            grnt_l_pep_note_typ = 488
            AND grnt_pep_id = gp.grnt_pep_id
    ) AS cdbo_send_vrftn_comment,
    (
        SELECT
            field_value
        FROM
            grnt41.grnt_l_pep_stts
        WHERE
            field_id = gp.grnt_l_pep_stts
    ) AS grnt_l_pep_stts_value,
    (
        SELECT
            field_value
        FROM
            grnt41.grnt_l_send_out_mthd
        WHERE
            field_id = gp.grnt_l_send_out_mthd
    ) AS grnt_l_send_out_mthd_value,
    (
        SELECT
            ldp.field_value
        FROM
            grnt41.grnt_aplctn   apl,
            grnt41.grnt_cntct    cicon,
            grnt.grnt_l_dept     ldp
        WHERE
            apl.grnt_aplctn_id = gp.grnt_aplctn_id
            AND cicon.cty_dept_yn = 1
            AND apl.grnt_city_dept_cntct_id = cicon.grnt_cntct_id
            AND cicon.grnt_l_dept = ldp.field_id
    ) AS department,
    grnt_l_rec_color,
    (
        SELECT
            field_value
        FROM
            grnt41.grnt_l_rec_color
        WHERE
            field_id = gp.grnt_l_rec_color
    ) AS grnt_l_rec_color_value,
    (
        SELECT
            grnt_user.grnt_utility.value_list('
                        select 
                            field_value 
                        from 
                            GRNT41.GRNT_PEP_CNCL_DIST A,
                            GRNT41.GRNT_L_CNCL_DIST B

                        where B.FIELD_ID=A.GRNT_L_CNCL_DIST
                        and
                        grnt_pep_id = '
            || gp.grnt_pep_id)
        FROM
            dual
    ) AS council_district,
    temp.status_date
FROM
    grnt41.grnt_pep gp,
    (
        SELECT
            MAX(mod_dt) AS status_date,
            pk_id
        FROM
            grnt38.grnt_hist
        WHERE
            table_nm = 'GRNT_PEP'
            AND pk_nm = 'GRNT_PEP_ID'
            AND field_nm = 'GRNT_L_PEP_STTS'
        GROUP BY
            pk_id
    ) temp
WHERE
    gp.grnt_pep_id = temp.pk_id (+)
    AND grnt_pep_id IN (
        SELECT
            grnt_pep_id
        FROM
            grnt41.grnt_pep
    )
ORDER BY
    proj_id,
    pep_agcy_nm""", engine)

df.head(5)

,pgm_year,grnt_pep_id,pep_agcy_nm,pep_proj_nm,citywide_cncl_dist_yn,grnt_l_pep_stts,pep_vrftn_dt,pep_to_environ_dt,pep_to_mgmt_dt,pep_send_out_dt,...,pep_from_environ_comment,pep_to_mgmt_comment,cdbo_send_vrftn_comment,grnt_l_pep_stts_value,grnt_l_send_out_mthd_value,department,grnt_l_rec_color,grnt_l_rec_color_value,council_district,status_date
0,41,418305,GENERAL ADMINISTRATION,AGING DEPARTMENT,NaN,530,04/16/2015,04/16/2015,05/13/2015,05/14/2015,...,None,None,None,Approved,None,"AGING, DEPARTMENT OF",4.0,Green,10,None
1,41,421925,AIDS POLICY DEVELOPMENT,AIDS POLICY DEVELOPMENT,1.0,530,06/17/2015,06/18/2015,07/02/2015,07/07/2015,...,None,None,None,Approved,None,"DISABILITY, DEPARTMENT ON",4.0,Green,None,None
2,41,420833,OFFICE OF THE CITY ATTORNEY,CITY ATTORNEY - CDBG ADMINISTRATION FOR HCID,1.0,530,05/18/2015,05/18/2015,05/22/2015,05/27/2015,...,None,None,None,Approved,None,CITY ATTORNEY,4.0,Green,None,None
3,41,420851,HCIDLA / COMMISSION FOR COMMUNITY & FAMILY SER...,COMMISSION FOR COMMUNITY AND FAMILY SERVICES,1.0,530,None,05/20/2015,06/03/2015,06/04/2015,...,None,None,None,Approved,None,HOUSING AND COMMUNITY INVESTMENT DEPARTMENT,4.0,Green,None,None
4,41,420661,ECONOMIC AND WORKFORCE DEVELOPMENT DEPARTMENT,ECONOMIC AND WORKFORCE DEVELOPMENT DEPARTMENT,1.0,530,05/12/2015,05/12/2015,05/22/2015,05/27/2015,...,None,None,None,Approved,None,ECONOMIC AND WORKFORCE DEVELOPMENT DEPARTMENT,4.0,Green,None,None


In [10]:
USER = "GRNT37"
engine.execute(f"SELECT table_name FROM all_tables WHERE owner='{USER}'").fetchall()

df = pd.read_sql(
    """SELECT
    '37' AS pgm_year,
    grnt_pep_id,
    pep_agcy_nm,
    pep_proj_nm,
    citywide_cncl_dist_yn,
    grnt_l_pep_stts,
    to_char(pep_vrftn_dt, 'mm/dd/yyyy') AS pep_vrftn_dt,
    to_char(pep_to_environ_dt, 'mm/dd/yyyy') AS pep_to_environ_dt,
    to_char(pep_to_mgmt_dt, 'mm/dd/yyyy') AS pep_to_mgmt_dt,
    to_char(pep_send_out_dt, 'mm/dd/yyyy') AS pep_send_out_dt,
    grnt_l_send_out_mthd,
    to_char(cdbo_send_vrftn_dt, 'mm/dd/yyyy') AS cdbo_send_vrftn_dt,
    to_char(pep_rcv_dt, 'mm/dd/yyyy') AS pep_rcv_dt,
    (
        SELECT
            field_value
        FROM
            grnt37.grnt_l_pep_typ
        WHERE
            field_id = gp.grnt_l_pep_typ
    ) AS grnt_l_pep_typ_value,
    (
        SELECT
            prjct_ttl
        FROM
            grnt37.grnt_aplctn
        WHERE
            grnt_aplctn_id = gp.grnt_aplctn_id
    ) AS prjct_ttl,
    cdbg_fnd_amt,
    (
        SELECT
            proj_id
        FROM
            grnt37.grnt_prpsd_proj
        WHERE
            grnt_aplctn_id = gp.grnt_aplctn_id
    ) AS proj_id,
    (
        SELECT
            pep_note
        FROM
            grnt37.grnt_pep_note
        WHERE
            grnt_l_pep_note_typ = 470
            AND grnt_pep_id = gp.grnt_pep_id
    ) AS trackingcomment,
    (
        SELECT
            pep_note
        FROM
            grnt37.grnt_pep_note
        WHERE
            grnt_l_pep_note_typ = 485
            AND grnt_pep_id = gp.grnt_pep_id
    ) AS pep_vrftn_comment,
    (
        SELECT
            pep_note
        FROM
            grnt37.grnt_pep_note
        WHERE
            grnt_l_pep_note_typ = 486
            AND grnt_pep_id = gp.grnt_pep_id
    ) AS pep_to_environ_comment,
    (
        SELECT
            pep_note
        FROM
            grnt37.grnt_pep_note
        WHERE
            grnt_l_pep_note_typ = 487
            AND grnt_pep_id = gp.grnt_pep_id
    ) AS pep_to_mgmt_comment,
    (
        SELECT
            pep_note
        FROM
            grnt37.grnt_pep_note
        WHERE
            grnt_l_pep_note_typ = 488
            AND grnt_pep_id = gp.grnt_pep_id
    ) AS cdbo_send_vrftn_comment,
    (
        SELECT
            field_value
        FROM
            grnt37.grnt_l_pep_stts
        WHERE
            field_id = gp.grnt_l_pep_stts
    ) AS grnt_l_pep_stts_value,
    (
        SELECT
            field_value
        FROM
            grnt37.grnt_l_send_out_mthd
        WHERE
            field_id = gp.grnt_l_send_out_mthd
    ) AS grnt_l_send_out_mthd_value,
    (
        SELECT
            ldp.field_value
        FROM
            grnt37.grnt_aplctn   apl,
            grnt37.grnt_cntct    cicon,
            grnt37.grnt_l_dept   ldp
        WHERE
            apl.grnt_aplctn_id = gp.grnt_aplctn_id
            AND cicon.cty_dept_yn = 1
            AND apl.grnt_city_dept_cntct_id = cicon.grnt_cntct_id
            AND cicon.grnt_l_dept = ldp.field_id
    ) AS department,
    grnt_l_rec_color,
    (
        SELECT
            field_value
        FROM
            grnt37.grnt_l_rec_color
        WHERE
            field_id = gp.grnt_l_rec_color
    ) AS grnt_l_rec_color_value,
    (
        SELECT
            grnt_user.grnt_utility.value_list('
                        select 
                            field_value 
                        from 
                            GRNT37.GRNT_PEP_CNCL_DIST A,
                            GRNT37.GRNT_L_CNCL_DIST B

                        where B.FIELD_ID=A.GRNT_L_CNCL_DIST
                        and
                        grnt_pep_id = '
            || gp.grnt_pep_id)
        FROM
            dual
    ) AS council_district
FROM
    grnt37.grnt_pep gp
WHERE
    grnt_pep_id IN (
        SELECT
            grnt_pep_id
        FROM
            grnt37.grnt_pep
    )
ORDER BY
    proj_id,
    pep_agcy_nm""", engine)

df.head(5)

,pgm_year,grnt_pep_id,pep_agcy_nm,pep_proj_nm,citywide_cncl_dist_yn,grnt_l_pep_stts,pep_vrftn_dt,pep_to_environ_dt,pep_to_mgmt_dt,pep_send_out_dt,...,pep_vrftn_comment,pep_to_environ_comment,pep_to_mgmt_comment,cdbo_send_vrftn_comment,grnt_l_pep_stts_value,grnt_l_send_out_mthd_value,department,grnt_l_rec_color,grnt_l_rec_color_value,council_district
0,37,339423,CITY OF LOS ANGELES DEPARTMENT OF AGING,AGING DEPARTMENT - ADMINISTRATION,1.0,477,06/13/2011,06/13/2011,06/14/2011,06/15/2011,...,None,None,None,None,Approved as submitted,None,"AGING, DEPARTMENT OF",4.0,Green,None
1,37,355963,DEPARTMENT ON DISABILITY,AIDS POLICY DEVELOPMENT,1.0,477,11/16/2011,11/16/2011,11/17/2011,11/23/2011,...,None,None,None,None,Approved as submitted,None,"DISABILITY, DEPARTMENT ON",4.0,Green,None
2,37,339357,LOS ANGELES CITY ATTORNEY,CITY ATTORNEY - CDBG ADMINISTRATION FOR CDD,1.0,477,04/11/2012,04/11/2012,04/16/2012,04/19/2012,...,04/10 PEP WAS RESUBMITTED,None,None,None,Approved as submitted,None,CITY ATTORNEY,4.0,Green,None
3,37,359437,COMMUNITY DEVELOPMENT DEPARTMENT,COMMISSION FOR COMMUNITY AND FAMILY SERVICES ...,1.0,477,04/04/2012,04/04/2012,04/05/2012,04/10/2012,...,None,None,None,None,Approved as submitted,None,COMMUNITY DEVELOPMENT DEPARTMENT,4.0,Green,None
4,37,339475,CDD,COMMUNITY DEVELOPMENT DEPARTMENT GENERAL ADMIN...,1.0,477,06/13/2011,06/13/2011,06/14/2011,06/15/2011,...,None,None,None,None,Approved as submitted,None,COMMUNITY DEVELOPMENT DEPARTMENT,4.0,Green,None
